In [ ]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions, Distributed

num_cores = parse(Int, ENV["SLURM_CPUS_PER_TASK"])
addprocs(12)

@everywhere begin
    
Lx=1000 
Ly=500
abun=1000
timepoints=10000
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
reps=collect(1:300)
disp=[20,25,30,35,40]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(reps,disp,methodrefs[:,:ind]))

    

function percsim(par::Tuple{Int64, Int64, Int64}) 
        
        rep=par[1]
        disps=par[2]
        methods=par[3]
        
        cd("C:\\Users\\mihir\\Documents\\landscapes")

        landscape=CSV.read(string("landscape_",rep,".csv"), DataFrame)

        ind=landscape[1,2]
        rangepar=landscape[1,3]    

        landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
        landscape[:,:rep]=fill(1,nrow(landscape))
        landscape[:,:rangepars]=fill(5,nrow(landscape))

        initial=filter(:pres => x-> x .== 1,landscape)
        initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

        CSV.write(string("initial_",rep,"_",disps,"_",methodrefs[methods,2],".csv"),initial)

        popmat=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
        popmat=Matrix(popmat[:,Not("y")])


        habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
        habmat=Matrix{Float64}(habmat[:,Not("y")])

        #Threshold for a suitable habitat
        mid=quantile(landscape[:,:soiltype],0.5)


        #Define dispersal probability as a function of #neighbors for all three dispersal methods
        maxnb=((2*disps+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        disp_list=([0;fill(1,length(neigh))],[0;0.5 .+ (neigh ./2)],[0;1.0 .- neigh])  

        for i in 1:100 

            samp=shuffle((findall(popmat.==1)))
            popmat[samp[1:ndeath]].=0

            j=0

            while j<nbirth

                x=sample(1:Ly)
                y=sample(1:Lx)

                if(popmat[x,y]==0)

                    xs=((x-disps):(x+disps))
                    xs=xs[0 .<xs .<=Ly]

                    ys=(y-disps):(y+disps)
                    ys=ys[0 .<ys .<=Lx]

                    nb=sum(popmat[xs,ys])+1

                    val=ceil(Int,habmat[x,y]-mid)*
                        ceil(Int,(disp_list[methods][nb]-rand()))

                    popmat[x,y]=val

                    j=j+val

                end
            end   

        end 

            popmat1=DataFrame(popmat,:auto)
            rename!(x-> strip(string(x), ['x']),popmat1)
            popmat1[:,:y].=1:500
            popmat1=stack(popmat1,1:1000)
            rename!(popmat1,:variable=> :x, :value => :pres)
            popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
            filter!(:pres => x-> x .== 1,popmat1)
            popmat1[:,:rep].=ind
            popmat1[:,:rangepar].=rangepar
            popmat1[:,:disp].=disps
            popmat1[:,:method].=methodrefs[methods,2]
            
            CSV.write(string("popmat_",rep,"_",disps,"_",methodrefs[methods,2],".csv"),popmat1)
    end
end
               


In [ ]:
pmap(percsim,pars)

In [ ]:
using Distributed

addprocs(12)

@everywhere begin

using  CSV, DataFrames, Random, LinearAlgebra, Distributions

Lx=128 
Ly=128
abun=100
timepoints=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
reps=collect(201:224)
disp=[20,25,30,35,40]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))


function percsim2(reps::Int64)
    
    landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",reps,".csv"), DataFrame)

    repl=landscape.rep[1] 
    rangepar=landscape.rangepars[1]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]
    
    CSV.write(string("C:\\Users\\mihir\\Documents\\landscapes2\\initial_",reps,".csv"),initial)

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix{Float64}(habmat[:,Not("y")])
    
    #Threshold for a suitable habitat
    mid=quantile(landscape[:,:soiltype],0.5)
    
    result=DataFrame()
    
    for i1 in 1:length(pars)
        
        disps=pars[i1][1]
        methods=pars[i1][2]
        
        #Define dispersal probability as a function of #neighbors for all three dispersal methods
        comp_dist=20
        maxnb=((2*comp_dist+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
        d=Normal(0,disps/2)
            
        popmat=init
            
        for i in 1:100
    
            samp=findall(popmat.==1)
            mort=habmat[samp]
            del=wsample(1:length(mort),mort,ndeath)
            popmat[samp[del]].=0  

            samp2=shuffle(findall(popmat.==1))

            j=1

            while (sum(popmat)<(abun))

                x=samp2[j][1]
                y=samp2[j][2]

                xs=((x-disps):(x+disps))
                xs=xs[0 .<xs .<=Ly]

                ys=(y-disps):(y+disps)
                ys=ys[0 .<ys .<=Lx]

                inds=collect(Iterators.product(xs,ys))

                vals=Float64[]            

                for k in 1:length(inds)

                    xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                    xss=xss[0 .<xss .<=Ly]

                    yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                    yss=yss[0 .<yss .<=Lx]

                    nb=sum(popmat[xss,yss])+1 

                    val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                        comp_list[methods][nb]*
                        (sign(1-popmat[inds[k][1],inds[k][2]])) 

                    push!(vals,val)

                end

                ind=wsample(1:length(vals),vals)

                popmat[inds[ind][1],inds[ind][2]]=1

                j+=1

                end

        end
        
        popmat1=DataFrame(popmat,:auto)
        rename!(x-> strip(string(x), ['x']),popmat1)
        popmat1[:,:y].=1:Ly
        popmat1=stack(popmat1,1:Lx)
        rename!(popmat1,:variable=> :x, :value => :pres)
        popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
        filter!(:pres => x-> x .== 1,popmat1)
        popmat1[:,:rep].=repl
        popmat1[:,:rangepar].=rangepar
        popmat1[:,:disp].=disps
        popmat1[:,:method].=methodrefs[methods,2]        
        append!(result,popmat1)
            
    end
        
    CSV.write(string("C:\\Users\\mihir\\Documents\\landscapes2\\final_",reps,".csv"),result)
end
end 
    
        
pmap(percsim2,reps)  


In [2]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions

    landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",201,".csv"), DataFrame)

    repl=landscape.rep[1] 
    rangepar=landscape.rangepars[1]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]
    
    CSV.write(string("C:\\Users\\mihir\\Documents\\landscapes2\\initial_",reps,".csv"),initial)

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix{Float64}(habmat[:,Not("y")])

LoadError: UndefVarError: abun not defined

In [13]:
using  CSV, DataFrames, Random, LinearAlgebra, Distributions

Lx=128 
Ly=128
abun=100
timepoints=100
nbirth=ndeath=convert(Int64,floor(abun*0.1))
    
#reps=collect(201:224)
disp=[3,4,6,10]
methodrefs=DataFrame(ind=[1, 2, 3], methods=["neutral","positive","negative"])
pars=collect(Iterators.product(disp,methodrefs[:,:ind]))



    
    landscape=CSV.read(string("C:\\Users\\mihir\\Documents\\landscapes\\landscape_",1,".csv"), DataFrame)

    landscape=landscape[landscape.x .<129 .&& landscape.y .<129 ,:]

    repl=landscape.rep[1] 
    rangepar=landscape.rangepars[1]    

    landscape[:,:pres]=shuffle([ones(Int8,abun);zeros(Int8,(nrow(landscape)-abun))])
    landscape[:,:rep]=fill(repl,nrow(landscape))
    landscape[:,:rangepars]=fill(rangepar,nrow(landscape))

    initial=filter(:pres => x-> x .== 1,landscape)
    initial=initial[:,[:rep,:rangepars,:x,:y,:pres]]

    init=unstack(landscape[:,["y","x","pres"]],:y,:x,:pres)
    init=Matrix(init[:,Not("y")])

    habmat=unstack(landscape[:,["y","x","soiltype"]],:y,:x,:soiltype)
    habmat=Matrix{Float64}(habmat[:,Not("y")])
    
    #Threshold for a suitable habitat
    mid=quantile(landscape[:,:soiltype],0.5)
    
    result=DataFrame()
    
    for i1 in 1:length(pars)
        
        disps=pars[i1][1]
        methods=pars[i1][2]
        
        #Define dispersal probability as a function of #neighbors for all three dispersal methods
        comp_dist=3
        maxnb=((2*comp_dist+1)^2)-1
        neigh=log.(maxnb,1:maxnb)
        comp_list=(fill(1,(length(neigh)+1)),[0.1; 0.1 .+ neigh],[1.0;1.0 .- neigh])    
        
        d=Normal(0,disps/2)
            
        popmat=init
            
        for i in 1:50
    
            samp=findall(popmat.==1)
            mort=habmat[samp]
            del=wsample(1:length(mort),mort,ndeath)
            popmat[samp[del]].=0  

            samp2=shuffle(findall(popmat.==1))

            j=1

            while (sum(popmat)<(abun))

                x=samp2[j][1]
                y=samp2[j][2]

                xs=((x-disps):(x+disps))
                xs=xs[0 .<xs .<=Ly]

                ys=(y-disps):(y+disps)
                ys=ys[0 .<ys .<=Lx]

                inds=collect(Iterators.product(xs,ys))

                vals=Float64[]            

                for k in 1:length(inds)

                    xss=((inds[k][1]-comp_dist):(inds[k][1]+comp_dist))
                    xss=xss[0 .<xss .<=Ly]

                    yss=(inds[k][2]-comp_dist):(inds[k][2]+comp_dist) 
                    yss=yss[0 .<yss .<=Lx]

                    nb=sum(popmat[xss,yss])+1 

                    val=pdf(d,sqrt(((x-inds[k][1])^2)+((y-inds[k][2])^2)))*                
                        comp_list[methods][nb]*
                        (sign(1-popmat[inds[k][1],inds[k][2]])) 

                    push!(vals,val)

                end

                ind=wsample(1:length(vals),vals)

                popmat[inds[ind][1],inds[ind][2]]=1

                j+=1
            
                end
        end
        
        popmat1=DataFrame(popmat,:auto)
        rename!(x-> strip(string(x), ['x']),popmat1)
        popmat1[:,:y].=1:Ly
        popmat1=stack(popmat1,1:Lx)
        rename!(popmat1,:variable=> :x, :value => :pres)
        popmat1[!,:x] = parse.(Int64,popmat1[!,:x])
        filter!(:pres => x-> x .== 1,popmat1)
        popmat1[:,:rep].=repl
        popmat1[:,:rangepar].=rangepar
        popmat1[:,:disp].=disps
        popmat1[:,:method].=methodrefs[methods,2]        
        append!(result,popmat1)
            
    end
        

 

In [10]:
result

AssertionError: AssertionError: Data frame is corrupt: length of column :disp (4) does not match length of column 1 (1200). The column vector has likely been resized unintentionally (either directly or because it is shared with another data frame).

In [14]:
CSV.write("res.csv",result)

"res.csv"